<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/final_LSTM_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np

In [44]:
FRIEND_DATA = list()
VOCAB_SIZE = 0

In [45]:
# Make the pair question and answer list
path = '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt'
with open(path) as frienddata:
  i = 0
  for data in frienddata:
    sent = data.split("__eou__")[:2]
    if len(sent) == 2:
      qaa = list()
      for subsent in sent:
        # Remove leading, ending space in string, " để mùa hạ qua từng đêm vắng  "  -> "để mùa hạ qua từng đêm vắng"
        subsent = subsent.strip()
        qaa.append(subsent)
      FRIEND_DATA.append(qaa)

In [46]:
FRIEND_DATA[:5]

[['Thích đánh lộn không?', 'Ngon nhà vô'],
 ['Solo yasua không', 'Chấp lun 2 mạng đầu'],
 ['Mai đi picnic không?', 'Mai bận học rồi'],
 ['Mai học ca mấy vậy?', 'Mai học ca 3'],
 ['Còn tiền không?', 'Còn chết liền']]

In [47]:
# Separate Answer and Question 
# Input text is question, target_text is answer
input_texts = list()
target_texts = list()
input_characters = set()
target_charaters = set()

for input_text, target_text in FRIEND_DATA:
  #make lower text
  input_text = input_text.lower()
  target_text = target_text.lower()
  target_text = '\t'+ target_text + '\n'

  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_charaters:
      target_charaters.add(char)
      

In [48]:
input_texts[:2]

['thích đánh lộn không?', 'solo yasua không']

In [49]:
target_texts[:2]

['\tngon nhà vô\n', '\tchấp lun 2 mạng đầu\n']

In [50]:
len(input_characters)

99

In [51]:
input_characters = sorted(list(input_characters))
target_charaters = sorted(list(target_charaters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_charaters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [53]:
print("Number of samples: ", len(input_texts))
print("Number of unique input tokens: ", num_encoder_tokens)
print("Number of unique output tokens: ", num_decoder_tokens)
print("Max sequence length for inputs: ", max_encoder_seq_length)
print("Max sequence length for outputs: ", max_decoder_seq_length)

Number of samples:  610
Number of unique input tokens:  99
Number of unique output tokens:  112
Max sequence length for inputs:  109
Max sequence length for outputs:  291


In [54]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_charaters)]
)

In [79]:
target_token_reverse_index = { value : key for key, value in target_token_index.items() }

In [58]:
{key:input_token_index[key] for i, key in enumerate(input_token_index) if i < 5}

{' ': 0, '"': 1, ',': 2, '.': 3, '/': 4}

In [106]:
{key:target_token_index[key] for i, key in enumerate(target_token_index) if i < 5}

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '(': 4}

In [60]:
# 3D array of shape (num_pairs, max_sentence_length_char, num_characters)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32'
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32'
)

In [63]:
encoder_input_data.shape, decoder_input_data.shape

((610, 109, 99), (610, 291, 112))

In [64]:
# build one hot vector
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1
  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1

  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1
    if t > 0:
      # decoder_target_data is ahead of decoder_input_data by one timestep
      # and will not include the start character /t
      decoder_target_data[i, t - 1, target_token_index[char]] = 1
  
  decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
  decoder_target_data[i, t:, target_token_index[' ']] = 1

In [ ]:
'''
  decoder_target_data is ahead of decoder_input_data by one timestep
  such as, decoder_input = /ttoi di hoc -> /t t o i d i h o c
           decoder_target               -> t o i d i h o c
'''

In [69]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 300

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 99)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 112)]  0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 300), (None, 480000      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 300),  495600      input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [71]:
batch_size = 100
epochs = 10
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/10
5/5 [==============================] - 19s 3s/step - loss: 3.3565 - accuracy: 0.5108 - val_loss: 0.4683 - val_accuracy: 0.9362
Epoch 2/10
5/5 [==============================] - 15s 3s/step - loss: 0.8529 - accuracy: 0.8730 - val_loss: 0.4641 - val_accuracy: 0.9363
Epoch 3/10
5/5 [==============================] - 15s 3s/step - loss: 0.8232 - accuracy: 0.8711 - val_loss: 0.4452 - val_accuracy: 0.9362
Epoch 4/10
5/5 [==============================] - 15s 3s/step - loss: 0.9245 - accuracy: 0.8522 - val_loss: 0.5565 - val_accuracy: 0.9142
Epoch 5/10
5/5 [==============================] - 15s 3s/step - loss: 0.8189 - accuracy: 0.8603 - val_loss: 0.4228 - val_accuracy: 0.9362
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 0.8252 - accuracy: 0.8724 - val_loss: 0.4203 - val_accuracy: 0.9360
Epoch 7/10
5/5 [==============================] - 15s 3s/step - loss: 0.7692 - accuracy: 0.8711 - val_loss: 0.4022 - val_accuracy: 0.9340
Epoch 8/10
5/5 [==================

In [105]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=20,
          validation_split=0.2)

Epoch 1/20
5/5 [==============================] - 16s 3s/step - loss: 0.6821 - accuracy: 0.8727 - val_loss: 0.3439 - val_accuracy: 0.9364
Epoch 2/20
5/5 [==============================] - 15s 3s/step - loss: 0.6548 - accuracy: 0.8724 - val_loss: 0.3384 - val_accuracy: 0.9366
Epoch 3/20
5/5 [==============================] - 15s 3s/step - loss: 0.6339 - accuracy: 0.8732 - val_loss: 0.3710 - val_accuracy: 0.9368
Epoch 4/20
5/5 [==============================] - 15s 3s/step - loss: 0.6423 - accuracy: 0.8732 - val_loss: 0.3306 - val_accuracy: 0.9374
Epoch 5/20
5/5 [==============================] - 15s 3s/step - loss: 0.6356 - accuracy: 0.8742 - val_loss: 0.4702 - val_accuracy: 0.9368
Epoch 6/20
5/5 [==============================] - 15s 3s/step - loss: 0.6522 - accuracy: 0.8740 - val_loss: 0.3207 - val_accuracy: 0.9366
Epoch 7/20
5/5 [==============================] - 15s 3s/step - loss: 0.6369 - accuracy: 0.8740 - val_loss: 0.3427 - val_accuracy: 0.9367
Epoch 8/20
5/5 [==================

In [110]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
5/5 [==============================] - 15s 3s/step - loss: 0.5465 - accuracy: 0.8760 - val_loss: 0.2728 - val_accuracy: 0.9376
Epoch 2/50
5/5 [==============================] - 15s 3s/step - loss: 0.5214 - accuracy: 0.8764 - val_loss: 0.2658 - val_accuracy: 0.9373
Epoch 3/50
5/5 [==============================] - 15s 3s/step - loss: 0.5205 - accuracy: 0.8770 - val_loss: 0.3601 - val_accuracy: 0.9249
Epoch 4/50
5/5 [==============================] - 15s 3s/step - loss: 0.7577 - accuracy: 0.8761 - val_loss: 0.2787 - val_accuracy: 0.9383
Epoch 5/50
5/5 [==============================] - 15s 3s/step - loss: 0.5203 - accuracy: 0.8779 - val_loss: 0.2701 - val_accuracy: 0.9387
Epoch 6/50
5/5 [==============================] - 15s 3s/step - loss: 0.5092 - accuracy: 0.8782 - val_loss: 0.2602 - val_accuracy: 0.9387
Epoch 7/50
5/5 [==============================] - 15s 3s/step - loss: 0.5024 - accuracy: 0.8786 - val_loss: 0.2668 - val_accuracy: 0.9408
Epoch 8/50
5/5 [==================

In [114]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
5/5 [==============================] - 15s 3s/step - loss: 0.3093 - accuracy: 0.9184 - val_loss: 0.1647 - val_accuracy: 0.9575
Epoch 2/50
5/5 [==============================] - 15s 3s/step - loss: 0.3069 - accuracy: 0.9191 - val_loss: 0.1653 - val_accuracy: 0.9585
Epoch 3/50
5/5 [==============================] - 15s 3s/step - loss: 0.3043 - accuracy: 0.9192 - val_loss: 0.1636 - val_accuracy: 0.9587
Epoch 4/50
5/5 [==============================] - 15s 3s/step - loss: 0.3026 - accuracy: 0.9192 - val_loss: 0.1629 - val_accuracy: 0.9582
Epoch 5/50
5/5 [==============================] - 15s 3s/step - loss: 0.3008 - accuracy: 0.9200 - val_loss: 0.1661 - val_accuracy: 0.9576
Epoch 6/50
5/5 [==============================] - 15s 3s/step - loss: 0.3004 - accuracy: 0.9202 - val_loss: 0.1607 - val_accuracy: 0.9590
Epoch 7/50
5/5 [==============================] - 15s 3s/step - loss: 0.2958 - accuracy: 0.9207 - val_loss: 0.1609 - val_accuracy: 0.9593
Epoch 8/50
5/5 [==================

In [122]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
5/5 [==============================] - 15s 3s/step - loss: 0.2198 - accuracy: 0.9380 - val_loss: 0.1541 - val_accuracy: 0.9597
Epoch 2/50
5/5 [==============================] - 15s 3s/step - loss: 0.2198 - accuracy: 0.9379 - val_loss: 0.1544 - val_accuracy: 0.9599
Epoch 3/50
5/5 [==============================] - 15s 3s/step - loss: 0.2160 - accuracy: 0.9397 - val_loss: 0.1549 - val_accuracy: 0.9596
Epoch 4/50
5/5 [==============================] - 15s 3s/step - loss: 0.2135 - accuracy: 0.9394 - val_loss: 0.1551 - val_accuracy: 0.9601
Epoch 5/50
5/5 [==============================] - 15s 3s/step - loss: 0.2595 - accuracy: 0.9313 - val_loss: 0.1608 - val_accuracy: 0.9598
Epoch 6/50
5/5 [==============================] - 15s 3s/step - loss: 0.2397 - accuracy: 0.9328 - val_loss: 0.1538 - val_accuracy: 0.9600
Epoch 7/50
5/5 [==============================] - 15s 3s/step - loss: 0.2316 - accuracy: 0.9346 - val_loss: 0.1523 - val_accuracy: 0.9608
Epoch 8/50
5/5 [==================

In [130]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2)

Epoch 1/100
5/5 [==============================] - 15s 3s/step - loss: 0.1391 - accuracy: 0.9620 - val_loss: 0.1663 - val_accuracy: 0.9594
Epoch 2/100
5/5 [==============================] - 15s 3s/step - loss: 0.1354 - accuracy: 0.9632 - val_loss: 0.1682 - val_accuracy: 0.9591
Epoch 3/100
5/5 [==============================] - 15s 3s/step - loss: 0.1332 - accuracy: 0.9643 - val_loss: 0.1676 - val_accuracy: 0.9596
Epoch 4/100
5/5 [==============================] - 15s 3s/step - loss: 0.1351 - accuracy: 0.9632 - val_loss: 0.1697 - val_accuracy: 0.9595
Epoch 5/100
5/5 [==============================] - 15s 3s/step - loss: 0.1288 - accuracy: 0.9658 - val_loss: 0.1702 - val_accuracy: 0.9586
Epoch 6/100
5/5 [==============================] - 15s 3s/step - loss: 0.1308 - accuracy: 0.9650 - val_loss: 0.1703 - val_accuracy: 0.9588
Epoch 7/100
5/5 [==============================] - 15s 3s/step - loss: 0.1273 - accuracy: 0.9658 - val_loss: 0.1714 - val_accuracy: 0.9592
Epoch 8/100
5/5 [==========

In [136]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=30,
          validation_split=0.2)

Epoch 1/30
5/5 [==============================] - 15s 3s/step - loss: 0.0442 - accuracy: 0.9895 - val_loss: 0.2247 - val_accuracy: 0.9580
Epoch 2/30
5/5 [==============================] - 15s 3s/step - loss: 0.0380 - accuracy: 0.9907 - val_loss: 0.2254 - val_accuracy: 0.9577
Epoch 3/30
5/5 [==============================] - 15s 3s/step - loss: 0.0382 - accuracy: 0.9905 - val_loss: 0.2281 - val_accuracy: 0.9576
Epoch 4/30
5/5 [==============================] - 15s 3s/step - loss: 0.0400 - accuracy: 0.9904 - val_loss: 0.2327 - val_accuracy: 0.9570
Epoch 5/30
5/5 [==============================] - 15s 3s/step - loss: 0.0437 - accuracy: 0.9894 - val_loss: 0.2304 - val_accuracy: 0.9570
Epoch 6/30
5/5 [==============================] - 15s 3s/step - loss: 0.0390 - accuracy: 0.9906 - val_loss: 0.2275 - val_accuracy: 0.9577
Epoch 7/30
5/5 [==============================] - 15s 3s/step - loss: 0.0359 - accuracy: 0.9910 - val_loss: 0.2302 - val_accuracy: 0.9577
Epoch 8/30
5/5 [==================

In [143]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
5/5 [==============================] - 15s 3s/step - loss: 0.0316 - accuracy: 0.9915 - val_loss: 0.2456 - val_accuracy: 0.9571
Epoch 2/50
5/5 [==============================] - 15s 3s/step - loss: 0.0447 - accuracy: 0.9879 - val_loss: 0.2370 - val_accuracy: 0.9574
Epoch 3/50
5/5 [==============================] - 15s 3s/step - loss: 0.0372 - accuracy: 0.9902 - val_loss: 0.2390 - val_accuracy: 0.9576
Epoch 4/50
5/5 [==============================] - 15s 3s/step - loss: 0.0305 - accuracy: 0.9916 - val_loss: 0.2398 - val_accuracy: 0.9581
Epoch 5/50
5/5 [==============================] - 15s 3s/step - loss: 0.0299 - accuracy: 0.9915 - val_loss: 0.2422 - val_accuracy: 0.9572
Epoch 6/50
5/5 [==============================] - 15s 3s/step - loss: 0.0300 - accuracy: 0.9915 - val_loss: 0.2452 - val_accuracy: 0.9572
Epoch 7/50
5/5 [==============================] - 15s 3s/step - loss: 0.0308 - accuracy: 0.9913 - val_loss: 0.2459 - val_accuracy: 0.9571
Epoch 8/50
5/5 [==================

In [151]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2)

Epoch 1/100
5/5 [==============================] - 15s 3s/step - loss: 0.0275 - accuracy: 0.9917 - val_loss: 0.2642 - val_accuracy: 0.9568
Epoch 2/100
5/5 [==============================] - 15s 3s/step - loss: 0.0334 - accuracy: 0.9902 - val_loss: 0.2596 - val_accuracy: 0.9565
Epoch 3/100
5/5 [==============================] - 15s 3s/step - loss: 0.0337 - accuracy: 0.9903 - val_loss: 0.2575 - val_accuracy: 0.9570
Epoch 4/100
5/5 [==============================] - 15s 3s/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 0.2585 - val_accuracy: 0.9572
Epoch 5/100
5/5 [==============================] - 15s 3s/step - loss: 0.0262 - accuracy: 0.9919 - val_loss: 0.2583 - val_accuracy: 0.9574
Epoch 6/100
5/5 [==============================] - 15s 3s/step - loss: 0.0261 - accuracy: 0.9917 - val_loss: 0.2607 - val_accuracy: 0.9567
Epoch 7/100
5/5 [==============================] - 15s 3s/step - loss: 0.0261 - accuracy: 0.9916 - val_loss: 0.2614 - val_accuracy: 0.9574
Epoch 8/100
5/5 [==========

In [161]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=300,
          validation_split=0.2)

Epoch 1/300
5/5 [==============================] - 15s 3s/step - loss: 0.0245 - accuracy: 0.9918 - val_loss: 0.2835 - val_accuracy: 0.9572
Epoch 2/300
5/5 [==============================] - 15s 3s/step - loss: 0.0244 - accuracy: 0.9918 - val_loss: 0.2874 - val_accuracy: 0.9569
Epoch 3/300
5/5 [==============================] - 15s 3s/step - loss: 0.0244 - accuracy: 0.9917 - val_loss: 0.2853 - val_accuracy: 0.9569
Epoch 4/300
5/5 [==============================] - 15s 3s/step - loss: 0.0249 - accuracy: 0.9917 - val_loss: 0.2845 - val_accuracy: 0.9574
Epoch 5/300
5/5 [==============================] - 15s 3s/step - loss: 0.0244 - accuracy: 0.9917 - val_loss: 0.2799 - val_accuracy: 0.9568
Epoch 6/300
5/5 [==============================] - 15s 3s/step - loss: 0.0247 - accuracy: 0.9917 - val_loss: 0.2844 - val_accuracy: 0.9573
Epoch 7/300
5/5 [==============================] - 15s 3s/step - loss: 0.0245 - accuracy: 0.9917 - val_loss: 0.2865 - val_accuracy: 0.9570
Epoch 8/300
5/5 [==========

In [162]:
# inference setup
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [166]:
encoder_model.save('encoder_model_760epochs.h5')

In [167]:
decoder_model.save("decoder_model_760epochs.h5")

In [170]:
model.save('model_760epochs.h5')

In [97]:
target_seq = np.zeros((1, 1, num_decoder_tokens))

In [98]:
target_seq

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [100]:
target_seq[0, 0, target_token_index['\t']] = 1.
target_seq

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [163]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        # print(output_tokens.shape) # 1 1 112
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_token_reverse_index[sampled_token_index]
        # print(sampled_char)
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [88]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [156]:
input_texts[1]

'solo yasua không'

In [164]:
for i in range(5):
  print(decode_sequence(np.array([encoder_input_data[i]])))

mình thích đi ăn cùng người yêu

mình thích đi ăn cùng người yêu

mình thích đi ăn cùng người yêu

mình thích đi ăn cùng người yêu

mình thích đi ăn cùng người yêu



In [ ]:
mình thích nghe nhạc hoặc xem phim, đôi khi là chơi game nha binh cùng khang cho tho tao sone với crush kết không thể mào tối bạn của sứnh trảnh cũng không biế tới với bạn bè thì là cũng chứa biết chá nhiền là chía chải tao mài lên 
